# imports

In [ ]:
import sys
sys.path.append('../')

import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

import json
import numpy as np
from models.patch_to_score.dataset import PatchToScoreCrossValidationDataset
from models.patch_to_score.models.modules.transformer_encoder_mlp import TransformerEncoderMLP
from models.patch_to_score.models.modules.global_sum_pooling import GlobalSumPooling

# constants

In [ ]:
model_results_dir_path = '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/results/patch_to_score/hypotheses/pts_shalllow_mlps_autoregressive_scannet/21_07/2025-08-08_22693c7950'
fold = 0

# load model

In [ ]:
with open(f'{model_results_dir_path}/configuration.json', 'rb') as f:
    configuration = json.load(f)
model_configuration = configuration['model']

In [ ]:
from models.patch_to_score.bootstrappers.model import build_model_from_configuration

In [ ]:
model = build_model_from_configuration(**model_configuration)
model.load_weights(f'{model_results_dir_path}/fold_{fold}/model.keras')

# analysis

In [ ]:
model.summary()

# inference

In [ ]:
cross_validation_dataset = PatchToScoreCrossValidationDataset(**configuration['data'])
dataset = cross_validation_dataset.fold_datasets[fold]
batch = [*dataset.validation_set, dataset.validation_sizes, dataset.validation_num_patch]

In [ ]:
predictions = model(batch)
saved_predictions = np.load(f'{model_results_dir_path}/fold_{fold}/validation/predictions.npy')

In [ ]:
np.allclose(predictions, saved_predictions)

In [ ]:
import pickle
def load_as_pickle(file_path):
    """
    Loads an object from a file using pickle.

    Parameters:
    file_path (str): The path to the file where the object is saved.

    Returns:
    any: The object loaded from the file.
    """
    with open(file_path, 'rb') as file:
        return pickle.load(file)

# all_predictions = load_as_pickle('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/21_07_with_pesto/folds_training_dicts.pkl')


In [ ]:
significances_and_str_patches = load_as_pickle('/home/iscb/wolfson/omriyakir/ubinet/results/patch_to_score/hypotheses/pts_shalllow_mlps_autoregressive_scannet/21_07/2025-08-08_22693c7950/candidates/significances_and_str_patches.pkl')

In [ ]:
print(significances_and_str_patches.keys())


In [ ]:
all_predictions_path = '/home/iscb/wolfson/omriyakir/ubinet/datasets/patch_to_score/data_for_training/21_07_with_pesto/folds_training_dicts.pkl'
proteins_path = '/home/iscb/wolfson/omriyakir/ubinet/datasets/patch_to_score/patches_dicts/21_07_with_pesto/merged_protein_objects.pkl'
all_predictions = load_as_pickle(all_predictions_path)
proteins = load_as_pickle(proteins_path)


In [ ]:
data_path = '/home/iscb/wolfson/omriyakir/ubinet/results/patch_to_score/hypotheses/pts_shalllow_mlps_autoregressive_scannet/21_07/2025-08-08_22693c7950/candidates/significances_and_str_patches.pkl'
data = load_as_pickle(data_path)
print(data.keys())

In [ ]:
import pandas as pd
path = '/home/iscb/wolfson/doririmon/home/order/bio-explore/data/esmc_layer_32_ubiquitin_classification/ensemble_test_predictions/end_epoch_6/all_folds.parquet'
df = pd.read_parquet(path)
uniprots = ['Q96PX1','O43294','Q96NL0']
df[df['uniprot'].isin(uniprots)]